In [79]:
import whoosh
import os
import csv
import sys

csv.field_size_limit(sys.maxsize)

9223372036854775807

In [80]:
protests = []
with open('protests.csv', 'rU') as csvfile: # open file
    reader = csv.DictReader(csvfile) # create a reader
    for row in reader: # loop through rows
        protests.append(row) # append each row to the list

In [81]:
#protests[:2]

In [82]:
type(protests)

list

In [83]:
keys = protests[1].keys()
keys

['', 'PUBLICATION', 'TITLE', 'COUNTRY', 'LENGTH', 'TEXT', 'DATE']

In [84]:
articles = [li['TEXT'] for li in protests]

In [95]:
#articles[:2]
len(articles)

1429

In [92]:
test_article = articles[339]
print test_article

Para o antropólogo Paulo Storani, ex-capitão do Batalhão de Operações Especiais (Bope) da PM, as autoridades de segurança estão perdendo a "guerra da comunicação" na defesa do programa das UPPs do Rio. Recentemente está se tornando comum protestos de moradores de favelas pacificadas exigindo a retirada das UPPs destes locais. Isso ocorreu nas várias manifestações de moradores do Pavão-Pavãozinho, em Copacabana (zona sul), contra a morte do dançarino do programa Esquenta, da TV Globo Douglas Rafael Pereira. "Nestes protestos estão surgindo várias faixas com os dizeres 'fora UPP'. Será que isso é uma exigência da maioria das pessoas que vivem ali, ou apenas de um grupo de moradores que ficaram sem renda com a saída dos traficantes? É sabido que muitas pessoas que não necessariamente estavam ligadas ao tráfico eram contratadas para fazer a endolação, ou seja, preparar a droga para venda", explica Storani. "Está faltando coragem de as autoridades assumirem os erros do projeto, talvez por s

In [115]:
test_article2 = articles[1229]
print test_article2

Os deputados estaduais de São Paulo aprovaram em votação simbólica, na noite de anteontem, um projeto que proíbe "o uso de máscara ou qualquer outro paramento que possa ocultar o rosto da pessoa, ou que dificulta ou impeça a sua identificação" nos protestos de rua. De autoria do deputado Campos Machado (PTB), o projeto de lei só exclui da proibição as "manifestações e as reuniões culturais no calendário oficial do Estado". O objetivo é inibir a atuação de manifestantes mascarados ligados à tática black bloc. Ele segue para o governador Geraldo Alckmin (PDSB), que tem o prazo de 15 dias para sancioná-lo ou vetá-lo. Ainda conforme o texto, a plataforma principal de reivindicação desse grupo é "destruir, danificar, explodir, queimar, saquear e aterrorizar". "Tal comportamento tem esvaziado as legítimas manifestações e prejudicado o direito dos demais cidadãos de bem de se manifestar. Além, por óbvio, de deixar rastros de pânico e destruição e, consequentemente, causando prejuízos ao erári

In [140]:
test_article3 = articles[189]
print test_article3

Já são quatro ônibus e três carros incendiados na tarde deste sábado em um protesto de cerca de 30 moradores do bairro Caramujo, em Niterói (cidade na Região Metropolitana), pela morte de Anderson Luiz Santos da Silva, de 21 anos, baleado na cabeça na madrugada de sexta-feira, 18. Os reflexos da manifestação já são sentidos na descida da Ponte Rio Niterói, via de acesso à Região dos Lagos, um dos principais destinos dos cariocas no feriadão. O protesto foi iniciado após o sepultamento de Anderson, hoje cedo. Os moradores acusam policiais militares de terem baleado o rapaz. Eles montaram barricadas e interditaram, com objetos em chamas, parte da Alameda São Boaventura, uma das principais ruas de Niterói, e da rodovia Amaral Peixoto. O Batalhão de Choque da Polícia Militar (BPChoque) já chegou ao local com cerca de cem homens. Um outro jovem morreu na manhã deste sábado em uma operação no bairro Morro do Céu, vizinho ao Caramujo. Emanoel Gomes, de 17 anos, saía de motocicleta de sua fest

In [146]:
#Rochelle's code that works in other notebook

def query_cities(article, city):

    text = article.split()
    if city in text:
        takeIndex = text.index(city)
        wc = len(text)
        if takeIndex > 20 and takeIndex < wc - 20:
            subset = ' '.join(text[takeIndex-20:takeIndex+20])
        elif takeIndex < 20 and takeIndex < wc - 20:
            subset = ' '.join(text[:takeIndex+20])
        elif takeIndex > 20 and takeIndex > wc - 20:
            subset = ' '.join(text[takeIndex-20:])

        if 'protest'or 'manifest' in subset:
            return(1)
    else:
        return(0)

In [147]:
query_cities(test_article, "Rio.")

1

In [148]:
query_cities(test_article,"Rio")

0

In [149]:
query_cities(test_article2, "Paulo")

1

In [150]:
query_cities(test_article2, "São Paulo")

0

In [151]:
query_cities(test_article2, "São")

1

In [152]:
query_cities(test_article3, "Rio")

1

In [157]:
for item in articles:
    print query_cities(item, "Rio")

0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
1
1
0
1
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
1
0
1
0
0
1
0
1
0
1
0
0
0
0
0
0
0
1
0
0
0
0
1
1
0
0
0
0
1
0
0
0
1
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
0
1
0
0
0
0
0
0
1
1
0
0
0
0
0
0
1
0
0
0
0
1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
1
1
1
0
0
0
0
0
1
0
0
0
0
0
0
1
0
0
0
1
0
1
0
0
0
0
0
0
0
1
1
0
0
0
1
0
0
0
0
0
0
0
0
1
0
0
0
1
1
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
1
1
0
0
0
0
0
1
0
0
0
0
0
0
1
0
1
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0


In [154]:
for d in articles:
    d['Rio'] = query_cities(d['TEXT'],"Rio")
    d['Curitiba'] = query_cities(d['TEXT'],"Curitiba")
    d['Belo Horizonte'] = query_cities(d['TEXT'],"Belo Horizonte")
    d['São Paulo'] = query_cities(d['TEXT'],"São Paulo")
    d['Salvador'] = query_cities(d['TEXT'],"Salvador")
    d['Porto Alegre'] = query_cities(d['TEXT'],"Porto Alegre")
    d['Fortaleza'] = query_cities(d['TEXT'],"Fortaleza")   
    d['Recife'] = query_cities(d['TEXT'],"Recife")
    d['Campinas'] = query_cities(d['TEXT'],"Campinas")
    d['Brasília'] = query_cities(d['TEXT'],"Brasília")

TypeError: string indices must be integers, not str